In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout
from keras.optimizers import SGD
from keras.utils import np_utils
import warnings; warnings.simplefilter('ignore')
import tensorflow as tf

Using TensorFlow backend.


In [12]:
data_dir = "/home/user/DeepLearning/projects/datasets/leaf_classification/"

In [13]:
train = pd.read_csv(data_dir+"/train.csv")
test = pd.read_csv(data_dir+"/test.csv")

In [16]:
def encode(train, test):
    label_encoder = LabelEncoder().fit(train.species)
    labels = label_encoder.transform(train.species)
    classes = list(label_encoder.classes_)

    train = train.drop(['species', 'id'], axis=1)
    test = test.drop('id', axis=1)

    return train, labels, test, classes

In [17]:
train, labels, test, classes = encode(train, test)

In [6]:
num_classes = train.species.value_counts().shape[0]
print("There are {} classes for the classification task.".format(num_classes))

There are 99 classes for the classification task.


In [22]:
# standardize train features
scaler = StandardScaler().fit(train.values)
scaled_train = scaler.transform(train.values)

In [23]:
# split train data into train and validation
sss = StratifiedShuffleSplit(test_size=0.1, random_state=23)
for train_index, valid_index in sss.split(scaled_train, labels):
    X_train, X_valid = scaled_train[train_index], scaled_train[valid_index]
    y_train, y_valid = labels[train_index], labels[valid_index]

In [24]:
nb_features = 64
X_train.shape ,X_valid.shape

((891, 192), (99, 192))

### number of features per features type (shape, texture, margin)

In [25]:
nb_class = len(classes)

In [26]:
# reshape train data
X_train_r = np.zeros((len(X_train), nb_features, 3))
X_train_r[:, :, 0] = X_train[:, :nb_features]
X_train_r[:, :, 1] = X_train[:, nb_features:128]
X_train_r[:, :, 2] = X_train[:, 128:]

In [29]:
# reshape validation data
X_valid_r = np.zeros((len(X_valid), nb_features, 3))
X_valid_r[:, :, 0] = X_valid[:, :nb_features]
X_valid_r[:, :, 1] = X_valid[:, nb_features:128]
X_valid_r[:, :, 2] = X_valid[:, 128:]

In [30]:
# Keras model with one Convolution1D layer
# unfortunately more number of covnolutional layers, filters and filters lenght 
# don't give better accuracy
model = Sequential()
model.add(Convolution1D(nb_filter=512, filter_length=1, input_shape=(nb_features, 3)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(nb_class))
model.add(Activation('softmax'))

In [31]:
y_train = np_utils.to_categorical(y_train, nb_class)
y_valid = np_utils.to_categorical(y_valid, nb_class)

In [32]:
sgd = SGD(lr=0.01, nesterov=True, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

In [33]:
nb_epoch = 15
model.fit(X_train_r, y_train, nb_epoch=nb_epoch, validation_data=(X_valid_r, y_valid), batch_size=16)

Train on 891 samples, validate on 99 samples
Epoch 1/15
891/891 [==============================] - 32s 36ms/step - loss: 3.6277 - accuracy: 0.2413 - val_loss: 1.4409 - val_accuracy: 0.6970
Epoch 2/15
891/891 [==============================] - 31s 35ms/step - loss: 0.6743 - accuracy: 0.8563 - val_loss: 0.3708 - val_accuracy: 0.8788
Epoch 3/15
891/891 [==============================] - 32s 36ms/step - loss: 0.1054 - accuracy: 0.9809 - val_loss: 0.1830 - val_accuracy: 0.9394
Epoch 4/15
891/891 [==============================] - 31s 35ms/step - loss: 0.0560 - accuracy: 0.9877 - val_loss: 0.2538 - val_accuracy: 0.9596
Epoch 5/15
891/891 [==============================] - 34s 38ms/step - loss: 0.1125 - accuracy: 0.9832 - val_loss: 0.1423 - val_accuracy: 0.9596
Epoch 6/15
891/891 [==============================] - 35s 40ms/step - loss: 0.0140 - accuracy: 0.9966 - val_loss: 0.1973 - val_accuracy: 0.9394
Epoch 7/15
891/891 [==============================] - 34s 38ms/step - loss: 0.0134 - accura